# REINFORCE Algorithm

### 1. Muestrar {$\tau^i$} de $\pi_{\theta}(a_t|s_t)$ - Correr M trayectorias usando la policy
### 2. Estimar el retorno: $$ R(\tau_i)  \approx \sum_{t=0}^{T}R(s_t^i, a_t^i)$$
### 3. Entrenar un modelo: $$ \nabla_{\theta} J_{\theta} \approx \frac{1}{M} \sum_{i=1}^{M}  R(\tau_i)   \sum_{t=0}^T \nabla_{\theta} log \pi_{\theta}(a_t^i|s_t^i)$$

# Implementación en Keras

### Suponiendo que solo corremos un episodio por iteración

La loss queda (Le ponemos un menos adelante para que tengamos que minimizar):

$$ \huge J_{\theta} =  - R \sum_{t=0}^T log \pi_{\theta}(a_t|s_t)$$
$$ \huge J_{\theta} =  - \sum_{t=0}^T log \pi_{\theta}(a_t|s_t) R $$
$$ \huge J_{\theta} =  \sum_{t=0}^T log \frac{1}{\pi_{\theta}(a_t|s_t)} R $$

Recordando la Entropía cruzada:

$$ \huge H(y_{true}, y_{pred}) = \sum_{i} y_{true_i} log (\frac{1}{y_{pred_i}}) $$

Ejemplo: 

- Sumpongamos que tenemos 3 acciones posibles y la red neuronal predijo $y_{pred}$ = [0.2, 0.3, 0.5]
- Se muestreó la salida y se eligión la acción 2, es decir la acción con probabilidad 0.3
- La $y_{true}$ será [0, 1, 0]

$$ \huge H = 0 log (\frac{1}{0.2}) + 1 log (\frac{1}{0.3}) + 0 log (\frac{1}{0.5}) $$

- Si redefinimos la $y_{true}$ como $y_{true}$ = $y_{true} R$
- La $y_{true}$ queda [0, R, 0]

$$ \huge H = 0 log (\frac{1}{0.2}) + R log (\frac{1}{0.3}) + 0 log (\frac{1}{0.5}) = R log (\frac{1}{\pi_{\theta}(a_t|s_t)}) $$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# conda install swig # needed to build Box2D in the pip install
# pip install box2d-py # a repackaged version of pybox2d

In [3]:
# conda install -c conda-forge tensorboardx

In [4]:
from REINFORCE_helper import BaseAgent, format_as_pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
import keras.backend as K
import numpy as np

In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()

In [6]:
class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        ## Defino métrica - loss normalizada (sin el retorno multiplicando)
        def loss_metric(y_true, y_pred):
            y_true_norm = K.sign(y_true)
            return K.categorical_crossentropy(y_true_norm, y_pred)
        model = Sequential()
        model.add(Dense(hidden_layer_neurons, input_shape=input_shape, activation='relu'))
        model.add(Dense(output_shape, activation='softmax'))

        model.compile(Adam(lr), loss=['categorical_crossentropy'], metrics=[loss_metric])
        return model
    
    def get_action(self, eval=False):
        p = self.model.predict([self.observation.reshape(1, self.nS)])
        if eval is False:
            action = np.random.choice(self.nA, p=p[0]) #np.nan_to_num(p[0])
        else:
            action = np.argmax(p[0])
        action_one_hot = np.zeros(self.nA)
        action_one_hot[action] = 1
        return action, action_one_hot, p
    
    def get_entropy(self, preds, epsilon=1e-12):
        entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return entropy
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

In [7]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=3)

In [8]:
reinforce_agent.reset_env()
action, action_one_hot, p = reinforce_agent.get_action()
print('Action:', action)
print('action_one_hot:', action_one_hot)
print('Policy prob dist:', p)

Action: 1
action_one_hot: [0. 1.]
Policy prob dist: [[0.50224835 0.49775165]]


In [24]:
reinforce_agent.reset_env()
obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs, time_step = reinforce_agent.get_experience_episodes(return_ts=True)

In [25]:
df = format_as_pandas(time_step, obs, preds, actions, rewards, disc_sum_rews, ep_returns, decimals = 3)

In [26]:
df.head(50)

,step,observation,policy_distribution,sampled_action,rewards,discounted_sum_rewards,episode_return
0,1,"[-0.045, -0.001, -0.005, -0.037]","[0.5, 0.499]","[1, 0]",1.0,16.864,16.864
1,2,"[-0.045, -0.196, -0.005, 0.253]","[0.474, 0.525]","[0, 1]",1.0,15.880,16.864
2,3,"[-0.049, 0.0, 0.0, -0.04]","[0.5, 0.499]","[0, 1]",1.0,14.895,16.864
3,4,"[-0.049, 0.194, -0.001, -0.333]","[0.51, 0.489]","[1, 0]",1.0,13.909,16.864
4,5,"[-0.045, 0.0, -0.008, -0.041]","[0.5, 0.499]","[0, 1]",1.0,12.922,16.864
5,6,"[-0.045, 0.194, -0.009, -0.337]","[0.509, 0.49]","[1, 0]",1.0,11.934,16.864
6,7,"[-0.041, 0.0, -0.015, -0.047]","[0.499, 0.5]","[0, 1]",1.0,10.945,16.864
7,8,"[-0.041, 0.194, -0.016, -0.345]","[0.509, 0.49]","[1, 0]",1.0,9.955,16.864
8,9,"[-0.037, 0.0, -0.023, -0.057]","[0.499, 0.5]","[0, 1]",1.0,8.964,16.864
9,10,"[-0.037, 0.195, -0.025, -0.357]","[0.509, 0.49]","[0, 1]",1.0,7.972,16.864


In [27]:
last_obs

array([ 0.02112008,  0.04944344, -0.21000621, -0.9093092 ])

In [13]:
reinforce_agent.get_entropy(preds)

0.99831235

# Algoritmo REINFORCE

In [6]:
from REINFORCE_helper import RunningVariance
from time import time

# Pruebas interesantes:
- n_experience_episodes=1, epochs=1, lr=0.001
- n_experience_episodes=5, epochs=1, lr=0.001
- n_experience_episodes=50, epochs=1, lr=0.001
- n_experience_episodes=50, epochs=20, lr=0.001
- n_experience_episodes=50, epochs=50, lr=0.001
- n_experience_episodes=50, epochs=50, lr=0.01

In [7]:
running_variance = RunningVariance()
running_variance.get_mean()

In [8]:
running_variance.get_mean()

In [ ]:
reinforce_agent = ReinforceAgent('CartPole-v1', n_experience_episodes=50, EPISODES=2000, 
                                 epochs=2, lr=0.001, gamma=0.999, batch_size=128, gif_to_board=True)
# reinforce_agent = ReinforceAgent('LunarLander-v2', n_experience_episodes=1, EPISODES=2000, epochs=1, lr=0.001)

running_variance = RunningVariance()
initial_time = time()


while reinforce_agent.episode < reinforce_agent.EPISODES:
    obs, actions, preds, disc_sum_rews, rewards, ep_returns, ep_len, last_obs = reinforce_agent.get_experience_episodes()
    for dr in ep_returns:
        running_variance.add(dr)
        
    pseudolabels = actions*ep_returns.reshape(-1, 1)

    history = reinforce_agent.model.fit(
        obs, pseudolabels, verbose=0, epochs=reinforce_agent.epochs, batch_size=128)
    
    reinforce_agent.log_data(reinforce_agent.episode, 
                      history.history['loss'][0], 
                      np.mean(ep_len), 
                      reinforce_agent.get_entropy(preds), 
                      running_variance.get_variance(), 
                      history.history['loss_metric'][0], 
                      time() - initial_time, np.mean(ep_returns[-1]),
                      rv_normalized = np.sqrt(running_variance.get_variance()/running_variance.get_mean()**2)
                    )

correr en linea de comando: tensorboard --logdir logs/
Episode: 51
Model on episode 52 improved from -inf to 146.21771449954537. Saved!
Episode: 103
Model on episode 104 improved from 146.21771449954537 to 147.92442528826075. Saved!
Episode: 1091
Model on episode 1092 improved from 147.92442528826075 to 197.56953313930887. Saved!
Episode: 1507